In [45]:
from nba_api.stats.endpoints import ShotChartDetail
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

from time import sleep

import psycopg2
from sqlalchemy import create_engine
import sqlconfig
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
#Getting cleaned data for use with the shotchart detail endpoint
def read_data():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )

        query = "SELECT b.game_id,p.player_id, p.team_id FROM nba24_25.active_nba_players p LEFT JOIN nba24_25.boxscores b ON p.player_id=b.player_id LIMIT 30;"

        try:
            data=pd.read_sql(query, engine)
            return data
        
        except Exception as e:
            print(f"An error occurred: {e}")
        

        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Fetching shotcharts details
def fetch_shotchart_data(player_id, game_id, team_id):
   
    try:
        shotchart = ShotChartDetail(player_id=player_id, game_id_nullable=game_id, team_id=team_id, context_measure_simple='FGA')
        fga = shotchart.shot_chart_detail.get_dict()  # Convert to dictionary
        print(f"Fetched data for Player {player_id}, Game {game_id}.")
        return fga
    except requests.exceptions.Timeout:
        print(f"Timeout for Player {player_id}, Game {game_id}. Skipping...")
    except requests.exceptions.RequestException as e:
        print(f"Request error for Player {player_id}, Game {game_id}: {e}")
    except Exception as e:
        print(f"Unexpected error for Player {player_id}, Game {game_id}: {e}")
    return None

def get_shotchart_data(data):
    fga_final = []
    for _, row in data.iterrows():
        fga = fetch_shotchart_data(player_id=row['player_id'], game_id=row['game_id'], team_id=row['team_id'])

        if fga and "data" in fga and "headers" in fga:
            FGA = pd.DataFrame(fga["data"], columns=fga["headers"])
            fga_final.append(FGA)
        else:
            print(f"Skipping Player {row['team_id']}, Game {row['game_id']} due to missing data.")
        sleep(1) 
        
    if fga_final:
        return pd.concat(fga_final, ignore_index=True)
    else:
        print("No data collected.")
        return pd.DataFrame()
    


In [ ]:
def normalization(data):#Normalization for BCNF
    #Game Events
    Game_events=data[['GAME_ID','GAME_EVENT_ID','PLAYER_ID','PERIOD','MINUTES_REMAINING','SECONDS_REMAINING','LOC_X','LOC_Y','SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG']]
    Game_events.columns=['game_id','event_id','player_id','current_period','minutes_remaining','seconds_remaining','loc_x','loc_y','shot_attempted','shot_made']
    Game_events=Game_events.copy()
    Game_events['game_id']=Game_events['game_id'].astype(str)
    Game_events['player_id']=Game_events['player_id'].astype(str)
    Game_events['event_id']=Game_events['event_id'].astype(str)

    print("Game Events")
    print(Game_events.head(5))

    #Shot Types 
    Shot_type=data[['LOC_X','LOC_Y','SHOT_TYPE']].drop_duplicates()
    Shot_type.columns=['loc_x','loc_y','shot_type']
    print("Shot Types")
    print(Shot_type.head(5))
    
    #Shot Zone Range
    Shot_zone_range=data[['LOC_X','LOC_Y','SHOT_ZONE_RANGE']].drop_duplicates()
    Shot_zone_range.columns=['loc_x','loc_y','shot_zone_range']
    print("Shot Zone Range")
    print(Shot_zone_range.head(5))

    #Shot Zone Area
    Shot_zone_area=data[['LOC_X','LOC_Y','SHOT_ZONE_AREA']].drop_duplicates()
    Shot_zone_area.columns=['loc_x','loc_y','shot_zone_area']
    print("Shot Zone Area")
    print(Shot_zone_area.head(5))

    #Shot Zone Basic
    Shot_zone_basic=data[['LOC_X','LOC_Y','SHOT_ZONE_BASIC']].drop_duplicates()
    Shot_zone_basic.columns=['loc_x','loc_y','shot_zone_basic']
    print("Shot Zone Basic")
    ']
    print(Shot_zone_basic.head(5))
    
    #Shot distance
    Shot_distance=data[['LOC_X','LOC_Y','SHOT_DISTANCE']].drop_duplicates()
    Shot_distance.columns=['loc_x','loc_y','shot_distance']
    print("Shot distance")
    print(Shot_distance.head(5))

    #For plotting NBA court
    data.to_csv("plotCourt.csv",index=False)

    return Game_events,Shot_type,Shot_zone_range,Shot_zone_area,Shot_zone_basic,Shot_distance

In [ ]:
def active_players(data,players):
    data=data[data['player_id'].isin(players['player_id'])]
    return data

In [50]:
def selectPlayers():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )

        query = "SELECT p.player_id FROM nba24_25.active_nba_players p;"

        try:
            data=pd.read_sql(query, engine)
            return data
        
        except Exception as e:
            print(f"An error occurred: {e}")
        

        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
def intitalInsertion(data,tablename):
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )
        
        # Insertions into the tables
        data.to_sql(tablename, engine, schema="nba24_25", if_exists="append", index=False)
      
        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
data=read_data()

In [ ]:
data=get_shotchart_data(data)

Fetched data for Player 1629060, Game 0022400835.
Fetched data for Player 2544, Game 0022400835.
Fetched data for Player 1629637, Game 0022400835.
Fetched data for Player 1630559, Game 0022400835.
Fetched data for Player 1629029, Game 0022400835.
Fetched data for Player 1627827, Game 0022400835.
Fetched data for Player 1629216, Game 0022400835.
Fetched data for Player 1629020, Game 0022400835.
Fetched data for Player 1630692, Game 0022400835.
Fetched data for Player 1642261, Game 0022400835.
Fetched data for Player 1642355, Game 0022400835.
Fetched data for Player 203458, Game 0022400835.
Fetched data for Player 1629003, Game 0022400835.
Fetched data for Player 202693, Game 0022400835.
Fetched data for Player 1629629, Game 0022400835.
Fetched data for Player 1641998, Game 0022400835.
Fetched data for Player 1628467, Game 0022400835.
Fetched data for Player 1631132, Game 0022400835.
Fetched data for Player 202691, Game 0022400835.
Fetched data for Player 1629023, Game 0022400835.
Fetche

In [53]:
players=selectPlayers()

In [54]:
Game_events,Shot_type,Shot_zone_range,Shot_zone_area,Shot_zone_basic,Shot_distance=normalization(data)

Game Events
      game_id event_id player_id current_period minutes_remaining  \
0  0022400835        9   1629060              1                11   
1  0022400835       44   1629060              1                 8   
2  0022400835       77   1629060              1                 5   
3  0022400835       95   1629060              1                 4   
4  0022400835      168   1629060              1                 1   

  seconds_remaining loc_x loc_y shot_attempted shot_made  
0                39     7    13              1         1  
1                30   105     4              1         0  
2                56  -194   159              1         0  
3                20   232    83              1         1  
4                15   -50   252              1         0  
Shot Types
  loc_x loc_y       shot_type
0     7    13  2PT Field Goal
1   105     4  2PT Field Goal
2  -194   159  3PT Field Goal
3   232    83  3PT Field Goal
4   -50   252  3PT Field Goal
Shot Zone Range
  loc_x loc_

In [55]:
Game_events=active_players(Game_events,players)

In [ ]:
intitalInsertion(Game_events,"game_events")
intitalInsertion(Shot_type,"shot_type")
intitalInsertion(Shot_zone_range,"shot_zone_range")
intitalInsertion(Shot_zone_area,"shot_zone_area")
intitalInsertion(Shot_zone_basic,"shot_zone_basic")
intitalInsertion(Shot_distance,"shot_distance")


Data inserted
